In [46]:
#Onging exploration:
#to explore: High: create a chat to show training journey in a pic.
#to explore: if j % 2 == 0:latest version, here needs to be re-visit, may impact final result.
#to explore: if possible to setup parameters under main(), so that from Master control script, Main() can directly called, instead of toggling between scripts.
#resolved - to explore: How to ensure AI's prediction is 2 in scenario like: [-1,-1,0,0,1,0,0,0,0]? Otherwise AI will lose immediatelly.
#resolved - to explore: should it be another model to explore all possible fastest lose situation? Or it is just put some noise will resolve
#to explore: High: add some noise into input/ output during training, so that AI can practice more scenarios. (some scenarios AI always loss)
#to explore: High: remove duplication on training data & score
#to explore: Ongoing: what should be a better DQ_ratio? Is it possible to be some fucntion instead of a constant value?

#lib:
import os.path
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
from shutil import copyfile

#import py lib from a different folder: https://www.fir3net.com/Programming/Python/how-do-you-import-a-python-module-from-another-folder.html
#self created lib in py file format
from Model.Game_Agent import game_agent
from Model.DL_Network_Model import Net
from Model.Funtion_Bank import data_input

#file_path_setup:
file_name_model_latest_version = 'Model/model_latest_version.pt'
file_name_model_last_version = 'Model/model_last_version.pt'
input_file_path = 'Data/training_data_input.csv'
score_file_path = 'Data/training_data_score.csv'
next_action_file_path = 'Data/training_data_next_action_taken.csv'
input_file_path_initiate = 'Data/training_data_input_initiate.csv'
score_file_path_initiate = 'Data/training_data_score_initiate.csv'
training_file_path = 'Data/training_data.csv'

#data format setup:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

#parameter setup:
#to explore: This part can be improved further.
#button = 1: RL model + predict via latest model
#button = 0: Only predict via latest model
button = 1
#train_model_from_crash = 1: train model from scratch
#train_model_from_crash = 0: reload saved model last time and continue with training.
train_model_from_crash = 0
main_loop_count = 20
epoch_size = 3000
steps_for_printing_out_loss = 1000
learning_rate = 0.2
DQ_ratio = 0.75


def Training_model():
    
    #indicator = 1 means there is an existing model saved
    #indicator = 0 means there is no model saved in folder
    if os.path.isfile(file_name_model_latest_version):
        indicator = 1
        input_data = data_input(input_file_path)
        score_data = data_input(score_file_path)
    else:
        indicator = 0
        input_data = data_input(input_file_path_initiate)
        score_data = data_input(score_file_path_initiate)
        #another option: give some initial value manually, for instance:
        #input_data = torch.tensor([0,0,0,0,0,0,0,0,0], dtype=torch.float)
        #score_data = torch.tensor([0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11], dtype=torch.float)

    input = input_data
    target = score_data
    net = Net()
    # 5 error functions with explainatioins(5 Regression Loss Functions All Machine Learners Should Know): https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0
    # HX: It should be alrighht to use MSE instead of MAE, as there is no outlier inside training data here
    loss_functioin = nn.MSELoss()
    # to explore: visulize parameter updating process, as well as loss impact.
    # optim comparation between: SGD & Adam: https://medium.com/@Biboswan98/optim-adam-vs-optim-sgd-lets-dive-in-8dbf1890fbdc
    # Here should use Adam compared with SGD(need to explore other Oprim as well), but learning rate should be setup with some strategy. --------Pending
    optimizer = optim.SGD(net.parameters(), lr = learning_rate)
    # to explore: if indicator == 0, any good start of parameter can be pre-assigned? instead of starting from scratch
    if indicator == 1:
        state_dict_last_version = torch.load('Model/model_latest_version.pt')['state_dict']
        net.load_state_dict(state_dict_last_version)
        #to explore function to load optimizer from existing model
        #optimizer.load_state_dict(torch.load('model_latest_version.pt')['optimizer'])
    #training loop:
    for i in range(1,epoch_size + 1):
        #to explore: a batch job for training
        #to explore: how to save best performance model during 6000 epochs
        optimizer.zero_grad()
        output = net(input)
        # this is the most important configuration: excludes calculation of loss without real target value
        output[target == -2] = -2
        output[target == -1.1] = -1.1
        # difference between loss, error,criterion: https://datascience.stackexchange.com/questions/10250/what-is-the-difference-between-objective-error-criterion-cost-loss-fun
        loss = loss_functioin(output, target)
        loss.backward()
        if i % (steps_for_printing_out_loss) == 0:
            print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
        # Does the update
        optimizer.step()
    
    if indicator == 1:
        copyfile(file_name_model_latest_version, file_name_model_last_version)
    else:
        torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_last_version)
    torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_latest_version)
    
    #to export prediction output & target, to review the major difference for optimizing model purpose
    # to explore: to visulize this part with real time if possible
    export_output = output.clone()
    export_output = export_output.cpu().detach().numpy()
    np.savetxt('Data/output.csv',export_output,delimiter=',')
    export_target = target.clone()
    export_target = export_target.cpu().detach().numpy()
    np.savetxt('Data/target.csv',export_target,delimiter=',')


def RL_model():
    tie = 0
    win = 0
    loss = 0
    not_valid_start = 0
    # pending for development: to let lastest to play with a same pre-defined version, to measure the performance.
    #load model
    model_last_version = Net()
    state_dict_last_version = torch.load(file_name_model_last_version)['state_dict']
    model_last_version.load_state_dict(state_dict_last_version)
    
    model_latest_version = Net()
    state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
    model_latest_version.load_state_dict(state_dict_latest_version)
    
    #load testing data: 82 starting points
    training_data = data_input(training_file_path)

    #to explore: is that possible to perform prediction below by batch, instead of 1 by 1?
    #game starts: between 2 versions
    for model_sequence in range(0, 2):
        for x in range(0,82):
            torch_tensor = training_data[x].clone()
            #initial
            input_status = []
            next_action_taken = []
            score = []
            for j in range(0,9):
                if j == model_sequence:
                    ###########another exceptional handling: if input without equal occurance of 1 & -1 - pending for coding
                    game_a = game_agent(torch_tensor,0)
                    if game_a.verify_result() == True:
                        print('not valid input, as game has been over')
                        not_valid_start += 1
                        break
                    elif 0 not in torch_tensor:
                        print('input tie & with 9 value')
                        not_valid_start += 1
                        break

                torch_tensor_saved = torch_tensor.clone()
                input_status.append(torch_tensor_saved.cpu().numpy())
                #predict
                #if j % 2 == 1, it means to let last version take the first action
                if j % 2 == model_sequence:
                    next_vision = model_latest_version(torch_tensor)
                else:
                    next_vision = model_last_version(torch_tensor)

                current_score = torch.ones(9) * -2
                #comprehance predict result, via making impossible option as '-1.1'
                for k in range(0,9):
                    if torch_tensor[k] != 0:
                        next_vision[k] = -1.1
                        current_score[k] = -1.1
                next_step = next_vision.argmax()
                #save next action

                next_action_taken.append(next_step.cpu().numpy())
                score.append(current_score.cpu().numpy())
                #generate result
                torch_tensor[next_step] = 1
                game_a = game_agent(torch_tensor,0)
                # varify status of result
                # training data duplication should be avoid somewhere, also some inout, action, and different DQ score????????
                if game_a.verify_result() == True:
                    # print(game_a.verify_result())
                    # calculate DQ
                    if j % 2 == 0:
                        win += 1
                    else:
                        loss += 1
                    # to explore: an enhancement on model, strategy to encourage model for a faster win 
                    current_final_score = 1
                    #update score
                    #To explore: DQ learning score enhancement? any alogrithm wise enhancement can be done??
                    for k in range(0, len(next_action_taken)):
                        score[-(k+1)][next_action_taken[-(k+1)]] = ((-1 * DQ_ratio)**(k))
                    break
                else:
                    if 0 not in torch_tensor:
                        #print('tie')
                        tie = tie + 1
                        current_final_score = 0
                        #update score
                        #To explore: DQ learning score enhancement? any alogrithm wise enhancement can be done??
                        for k in range(0, len(next_action_taken)):
                            score[-(k+1)][next_action_taken[-(k+1)]] = ((-1 * DQ_ratio)**(k))
                        break
                    else:
                        #generate next input
                        torch_tensor *= -1
            input_status_df = pd.DataFrame(input_status)
            input_status_df.to_csv(input_file_path, index=False, mode='a', header=False)
            next_action_taken_df = pd.DataFrame(next_action_taken)
            next_action_taken_df.to_csv(next_action_file_path, index=False, mode='a', header=False)
            score_df = pd.DataFrame(score)
            score_df.to_csv(score_file_path, index=False, mode='a', header=False)
    #remove duplication:
    input_status_df = pd.read_csv(input_file_path, header = None)
    next_action_taken_df = pd.read_csv(next_action_file_path, header = None)
    score_df = pd.read_csv(score_file_path, header = None)
    consul_df = pd.concat([input_status_df, next_action_taken_df, score_df], axis=1)
    consul_df = consul_df.replace(-0.0, 0.0)
    consul_df.drop_duplicates(keep = 'first', inplace = True) 
    input_status_df = consul_df.iloc[:,0:9].copy()
    next_action_taken_df = consul_df.iloc[:,9:10].copy()
    score_df = consul_df.iloc[:,10:].copy()
    input_status_df.to_csv(input_file_path, index=False, header=False)
    next_action_taken_df.to_csv(next_action_file_path, index=False, header=False)
    score_df.to_csv(score_file_path, index=False, header=False)

    print("win: " + str(win))
    print("Loss: " + str(loss))
    print("Tie: " + str(tie))
    print("Not valid start: " + str(not_valid_start))


if __name__ == "__main__":
    if train_model_from_crash == 1:
        os.remove(file_name_model_latest_version)
        os.remove(file_name_model_last_version)
        os.remove(input_file_path)
        os.remove(score_file_path)
        os.remove(next_action_file_path)
        
    if button == 1:
        for loops in range(0, main_loop_count):
            Training_model()
            RL_model()
    else:
        RL_model()

Loss (epoch: 1000): 0.015108448
Loss (epoch: 2000): 0.014349725
Loss (epoch: 3000): 0.014410454
win: 106
Loss: 48
Tie: 10
Not valid start: 0
Loss (epoch: 1000): 0.014300729
Loss (epoch: 2000): 0.0135454405
Loss (epoch: 3000): 0.013508496
win: 108
Loss: 45
Tie: 11
Not valid start: 0
Loss (epoch: 1000): 0.013603855
Loss (epoch: 2000): 0.013132577
Loss (epoch: 3000): 0.012697116
win: 115
Loss: 42
Tie: 7
Not valid start: 0
Loss (epoch: 1000): 0.01332021
Loss (epoch: 2000): 0.012273604
Loss (epoch: 3000): 0.013424181
win: 112
Loss: 42
Tie: 10
Not valid start: 0
Loss (epoch: 1000): 0.01251639
Loss (epoch: 2000): 0.012929315
Loss (epoch: 3000): 0.011715661
win: 110
Loss: 44
Tie: 10
Not valid start: 0
Loss (epoch: 1000): 0.012509066
Loss (epoch: 2000): 0.011721402
Loss (epoch: 3000): 0.011359464
win: 108
Loss: 47
Tie: 9
Not valid start: 0
Loss (epoch: 1000): 0.011424413
Loss (epoch: 2000): 0.01182347
Loss (epoch: 3000): 0.010969579
win: 109
Loss: 42
Tie: 13
Not valid start: 0
Loss (epoch: 1000